# Setup

In [1]:
import googlemaps
from datetime import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import requests
from PIL import Image
import time
from tqdm import tqdm
import json
import geopy.distance
from itertools import chain

with open('apikey.txt') as file:
    API_KEY = file.read().strip()

google_calls = 0
prev_lookups = {}

In [2]:
gmaps = googlemaps.Client(key=API_KEY)

with open('stoplocations.json') as file:
    bus_stop_locations = json.load(file)
with open("stoplookup.json") as file:
    stop_lookup = json.load(file)
with open('stopschedule.json') as file:
    stopschedule = json.load(file)
with open('stopnames.json') as file:
    stop_names = json.load(file)
with open('stoptimes.json') as file:
    stoptimes = json.load(file)




# stopschedule = {}
# for line, v in stopschedule_.items():
#     tmp1 = {}
#     for day in ['Aug 22', 'Aug 23', 'Aug 24', 'Aug 25', 'Aug 26']:
#         for tripid, v2 in v.get(day, {}).items():
#             tmp2 = {}
#             for stop, time in v2:
#                 tmp2[stop] = time
#             tmp1[tripid] = tmp2
#     stopschedule[line] = tmp1
    

In [7]:

def l1_geodesic(a,b):
    a = (a['lat'], a['lng'])
    b = (b['lat'], b['lng'])
    return geopy.distance.geodesic((a[0],a[1]), (a[0],b[1])).miles + geopy.distance.geodesic((a[0],b[1]), (b[0],b[1])).miles    


def l2_geodesic(a,b):
    a = (a['lat'], a['lng'])
    b = (b['lat'], b['lng'])
    return geopy.distance.geodesic(a,b).miles


def tripinfo(start, bus_start, bus_end, end):
    bike_dist = l1_geodesic(start, bus_stop_locations[bus_start])
    bus_dist = l1_geodesic(bus_stop_locations[bus_start], bus_stop_locations[bus_end])
    bike_dist2 = l1_geodesic(bus_stop_locations[bus_end], end)
    return bike_dist, bus_dist, bike_dist2

def time2float(time):
    hour, minute, second = list(map(int, time.split(':')))
    return hour + minute/60 + second/3600

def bike_distance(start, end):
    global prev_lookups
    tup = (start['lat'], start['lng'], end['lat'], end['lng'])
    if tup in prev_lookups:
        return prev_lookups[tup]
    global google_calls
    
    google_calls += 1
    
    d = gmaps.directions(start, end, mode='bicycling')[0]['legs'][0]['distance']['value'] / 1609
    prev_lookups[tup] = d
    return d
    
class DistanceList:
    def __init__(self, peg1, peg2, trip_id, stops, bus_weight=5, prefilter=True):
        self.peg1 = peg1
        self.peg2 = peg2
        self.trip_id = trip_id
        self.bus_weight = bus_weight
        start_dists = [(s1, l2_geodesic(peg1, bus_stop_locations[s1])) for s1 in stops]
        end_dists = [(s2, l2_geodesic(bus_stop_locations[s2], peg2)) for s2 in stops]
        self.data = [ (s1, s2, d1, d2, bus_distance(trip_id, s1, s2), False) for i,(s1,d1) in enumerate(start_dists) for s2,d2 in end_dists[i+1:]]
        self.key = lambda x: x[2] + x[3] + x[4]/self.bus_weight
        
        if prefilter:
            old_len = len(self.data)
            self.data = [self.data[i] for i in np.argpartition([self.key(x) for x in self.data], 10)[:10]]
#             print("data length reduced to ", len(self.data), " from ", old_len)
            
    def pop(self):
        i  = np.argmin([self.key(x) for x in self.data])
        while not self.data[i][-1]:
            sid1 = self.data[i][0]
            sid2 = self.data[i][1]
            self.data[i] = (sid1, sid2, bike_distance(self.peg1, bus_stop_locations[sid1]), bike_distance(bus_stop_locations[sid2], self.peg2), self.data[0][-2], True)
#             self.data = self.data[np.argpartition([self.key(x) for x in self.data], 0)]
            i  = np.argmin([self.key(x) for x in self.data])
        i  = np.argmin([self.key(x) for x in self.data])
        res = self.data.pop(i)
        return res[:-1]
    
def nearest_stops_on_trip(trip_id, stop_seq, start, end, timer=None):
    if timer is not None:
        timer.start('key creation')
        keys = sorted(list(stop_seq.keys()), key=lambda x: time2float(stop_seq[x]))
        timer.stop('key creation')
        timer.start('list creation')
    prelim_dists = DistanceList(start, end, trip_id, keys)
    if timer is not None:
        timer.stop('list creation')
        timer.start('pop')
    res= prelim_dists.pop() # bus_start, bus_end, start_dist, end_dist, bus_dist
    if timer is not None:
        timer.stop('pop')
    return res
    

def bus_distance(trip_id, start, end):
    return (stoptimes[trip_id][end] - stoptimes[trip_id][start]) / 1.609
#     trip = stoptimes[stoptimes['trip_id'] == trip_id]
#     start_dist = trip[trip['stop_id']==start]['shape_dist_traveled'].iloc[0]
#     end_dist = trip[trip['stop_id']==end]['shape_dist_traveled'].iloc[0]
#     if np.isnan(start_dist):
#         start_dist = 0
#     if np.isnan(end_dist):
#         end_dist = 0
#     return (end_dist - start_dist) / 1.609


In [8]:
from time import time as time_now
class Timer:
    def __init__(self):
        self.data = {}
    def start(self, key):
        self.data[key] =self.data.get(key, 0) - time_now()
    def stop(self, key):
        self.data[key] += time_now()
    def report(self):
        return
        for k, v in self.data.items():
            print(k, ': ', v)

# Script

In [9]:
start_address = "Comanche and Juan Tabo, Albuquerque, New Mexico"
end_address = "University of New Mexico, Albuquerque, New Mexico"

start_time = 7
end_time = 9


In [10]:
#
# First, check how long the biking distance would be
# (do that)

# Next, find all stops within 3 miles of the start address, and all stops within 3 miles of the end address
start_location = gmaps.geocode(start_address)[0]['geometry']['location']
end_location = gmaps.geocode(end_address)[0]['geometry']['location']


stops_close_to_start = [ stop for stop, coord in bus_stop_locations.items() if l1_geodesic(coord, start_location) < 3 ]
stops_close_to_end = [ stop for stop, coord in bus_stop_locations.items() if l1_geodesic(coord, end_location) < 3 ]

lines_close_to_start = {}
for stop in stops_close_to_start:
    for line in stop_lookup[stop]:
        if line not in lines_close_to_start:
            lines_close_to_start[line] = set()
        lines_close_to_start[line].add(stop)
        
        
lines_close_to_end = {}
for stop in stops_close_to_end:
    for line in stop_lookup[stop]:
        if line not in lines_close_to_end:
            lines_close_to_end[line] = set()
        lines_close_to_end[line].add(stop)
        
shared_lines = [k for k in lines_close_to_end if k in lines_close_to_start]        
bus_routes = {}
trips_wtimes = []
prev_calcs = {}

for line in tqdm(shared_lines):
    all_trips = stopschedule[str(line)]
    seen_trips = set()
    timer = Timer()
    for trip_id, stop_seq in all_trips.items():
        timer.start('initial check')
        if min(map(time2float, stop_seq.values())) > end_time or max(map(time2float, stop_seq.values())) < start_time:
            timer.stop('initial check')
            continue
        timer.stop('initial check')
        timer.start('tuple creation')
        tup = tuple(sorted(stop_seq.keys())) 
        timer.stop('tuple creation')
        timer.start('prev calcs lookup')
        if tup in prev_calcs:
            timer.stop('prev calcs lookup')
            timer.start('prev calcs fetch')
            bus_start, bus_end, start_dist, end_dist, bus_dist = prev_calcs[tup]
            timer.stop('prev calcs fetch')
        else:
            timer.stop('prev calcs lookup')
            timer.start('nearest stops calc')
            bus_start, bus_end, start_dist, end_dist, bus_dist = nearest_stops_on_trip(trip_id, stop_seq, start_location, end_location, timer=timer)
            timer.stop('nearest stops calc')
            timer.start('prev calcs store')
            prev_calcs[tup] = bus_start, bus_end, start_dist, end_dist, bus_dist
            timer.stop('prev calcs store')
        timer.start('the rest')
        time_start = time2float(stop_seq[bus_start])
        time_end = time2float(stop_seq[bus_end])
        if time_start > time_end:
            timer.stop('the rest')
            continue
        if time_start >= start_time and time_start < end_time and (time_start, time_end) not in seen_trips:
            seen_trips.add((time_start, time_end))
            trips_wtimes.append({'line': line, 'bike1': start_dist, 'bike2': end_dist, 'bus_dist': bus_distance(trip_id, bus_start, bus_end), \
                            'bus_start': bus_start, 'bus_end': bus_end, 'time_start': stop_seq[bus_start], 'time_end': stop_seq[bus_end], 'bus_duration': time_end - time_start})
        timer.stop('the rest')
    timer.report()
    if len(seen_trips) == 0:
        raise Exception("Uh oh stinky")
bike_only = bike_distance(start_location, end_location)
print(f"Travel from {start_address} to {end_address}")
print()
print(f"Bike directly: {round(bike_only, 3)} miles")
print()
for tinfo in sorted(trips_wtimes, key=lambda x: time2float(x['time_start'])):
    if tinfo['bike1'] + tinfo['bike2'] > bike_only *2/3:
        continue
    print(f"Take bus #{tinfo['line']}")
    print(f"\tBike {round(tinfo['bike1'],3)} miles to {stop_names[tinfo['bus_start']]}")
    print(f"\tBus at {tinfo['time_start']} for {round(tinfo['bus_dist'],3)} miles to {stop_names[tinfo['bus_end']]} at {tinfo['time_end']}")
    print(f"\tBike {round(tinfo['bike2'],3)} miles")
    print(f"Total biking: {round(tinfo['bike1'] + tinfo['bike2'], 3)}")
    print(f"Total distance: {round(tinfo['bike1']+tinfo['bike2']+tinfo['bus_dist'],3)}")
    print()                

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.51s/it]

Travel from Comanche and Juan Tabo, Albuquerque, New Mexico to University of New Mexico, Albuquerque, New Mexico

Bike directly: 9.287 miles

Take bus #8
	Bike 1.045 miles to Menaul @ Juan Tabo
	Bus at  7:02:00 for 6.886 miles to Menaul @ Vassar at  7:27:46
	Bike 2.661 miles
Total biking: 3.707
Total distance: 10.593

Take bus #12
	Bike 2.509 miles to Constitution @ Morris
	Bus at  7:07:22 for 7.173 miles to Lomas @ UNM Hospital at  7:36:00
	Bike 0.433 miles
Total biking: 2.942
Total distance: 10.115

Take bus #5
	Bike 0.542 miles to Montgomery @ Juan Tabo
	Bus at  7:17:00 for 8.703 miles to Lomas @ UNM Hospital at  7:46:00
	Bike 0.433 miles
Total biking: 0.975
Total distance: 9.678

Take bus #7
	Bike 0.59 miles to Candelaria @ Juan Tabo
	Bus at  7:18:00 for 5.463 miles to Candelaria @ Girard at  7:38:39
	Bike 3.043 miles
Total biking: 3.633
Total distance: 9.096

Take bus #11
	Bike 3.069 miles to Lomas @ Morris
	Bus at  7:22:48 for 5.133 miles to Lomas @ UNM Hospital at  7:40:00
	Bike